# Optimizer

## 梯度下降 (Gradient Descent)

我们的目标是找到一组参数 $\theta$，使得损失函数 $L(\theta)$ 最小化。假设我们对参数进行微小的更新 $\Delta \theta$，根据泰勒展开的一阶近似：

$$
L(\theta + \Delta \theta) \approx L(\theta) + \nabla L(\theta)^T \Delta \theta
$$

其中 $\nabla L(\theta)$ 是损失函数对参数的梯度。

为了让损失函数下降（即 $L(\theta + \Delta \theta) < L(\theta)$），我们需要：

$$
\nabla L(\theta)^T \Delta \theta < 0
$$

根据向量点积公式 $\vec{a} \cdot \vec{b} = \|\vec{a}\| \|\vec{b}\| \cos(\alpha)$，当 $\Delta \theta$ 与梯度 $\nabla L(\theta)$ 的方向相反（即夹角为 $180^\circ$, $\cos(\alpha) = -1$）时，点积最小（最负）。因此，我们取：
$$
\Delta \theta = -\eta \nabla L(\theta)
$$
其中 $\eta > 0$ 是学习率。这样能保证在步长足够小的情况下，损失函数值必然下降。

```{tip}
1. 随机选择初始参数 $\theta_0$。
2. 计算所有样本上的平均梯度 $g_t = \frac{1}{N} \sum_{i=1}^{N} \nabla L(x_i, \theta_t)$。
3. 沿梯度的反方向更新参数：$\theta_{t+1} = \theta_t - \eta g_t$。
4. 重复步骤 2-3。
```

## Mini-batch GD

上文描述的是全量梯度下降 (Batch Gradient Descent)，即每次更新都计算所有样本的梯度。但在深度学习中，我们常用的是 Mini-batch Gradient Descent。为什么要使用 Mini-batch？

1.  **显存限制**：LLM 的训练数据量巨大，不可能一次性将所有数据装入 GPU 显存来计算梯度。

2.  **计算效率与收敛速度**：
    *   全量梯度下降要把所有数据扫一遍才能更新一次参数，速度极慢。Mini-batch 每扫过一小批数据就更新一次，收敛更快。
    *   相比单样本更新（Stochastic Gradient Descent, Batch Size=1），Mini-batch 可以利用 GPU 的矩阵运算进行并行加速。

3.  **泛化能力 (Generalization)**：
    *   Mini-batch 的梯度是全量梯度的“有偏估计”，引入了随机噪声。
    *   这种噪声有助于模型跳出局部极小值 (Local Minima) 或鞍点 (Saddle Points)，找到泛化性更好的平坦极小值。


```{tip}
* 随机选择初始参数 $\theta_0$。
* 将训练集 $D$ 随机打乱，并划分为多个大小为 $B$ 的小批次 (Batch) $\{b_1, b_2, ..., b_M\}$。
* For epoch = 1 to N:
    * For batch $b_i$ in batches:
        1.  计算梯度：只计算当前 batch 上的平均梯度 $g_t = \frac{1}{B} \sum_{x \in b_i} \nabla L(x, \theta)$。
        2.  更新参数：$\theta \leftarrow \theta - \eta g_t$。
```


## 动量法 (Momentum)

想象一个小球从山上滚下来：
1.  动量：小球在滚动的过程中会积累速度。如果它一直在下坡，它会越滚越快。
2.  平滑路径：如果路面坑坑洼洼，小球因为有惯性，不会被小坑卡住，而是会冲过去；也不会因为路面突然变平就立刻停下来。

普通的梯度下降就像是一个人在走路，每一步都只看脚下的路（当前梯度），如果不陡了就立刻停下。而动量法就像是一个沉重的铁球，它不仅受当前坡度影响，还保留了之前的速度。

```{tip}
$$
\begin{aligned}
v_{t+1} &= \beta v_t + g_t \\
\theta_{t+1} &= \theta_t - \eta v_{t+1}
\end{aligned}
$$
```

## RMSprop

RMSprop 的核心思想是：**自适应学习率**。

想象你在滑雪：
1.  平缓区域：坡度很小，如果步子太小会走得很慢。RMSprop 会自动增大步长，让你跑快点。
2.  陡峭区域：坡度很大，如果步子太大容易滚下去或震荡。RMSprop 会自动减小步长，让你走稳点。


```{tip}
1.  衡量最近梯度的剧烈程度：
    $$ S_t = \beta S_{t-1} + (1 - \beta) g_t^2 $$
    *   $S_t$：梯度平方的累积量。
    *   $\beta$：衰减率。

2.  更新参数（梯度除以 $\sqrt{S_t}$）：
    $$ \theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{S_t + \epsilon}} g_t $$
    *   如果 $g_t$ 很大，$S_t$ 就很大，$\frac{1}{\sqrt{S_t}}$ 就很小 -> **步长变小，防止震荡**。
    *   如果 $g_t$ 很小，$S_t$ 就很小，$\frac{1}{\sqrt{S_t}}$ 就很大 -> **步长变大，加速收敛**。
```

## Adam

Adam 的核心思想非常简单：**把 Momentum 和 RMSprop 结合起来**。

1.  它记录梯度的“惯性”，让模型在正确的方向上加速。
2.  它记录梯度平方的“剧烈程度”，给每个参数自适应地调整学习率。

```{note}
1.  **计算动量（一阶矩）**：$m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t$ （类似于 Momentum 的 $v_t$）
2.  **计算能量（二阶矩）**：$v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2$ （类似于 RMSprop 的 $S_t$）
3.  **偏差修正**：因为 $m_0$ 和 $v_0$ 初始化为 0，刚开始会偏向 0。Adam 做了修正：
    *   $\hat{m}_t = m_t / (1 - \beta_1^t)$
    *   $\hat{v}_t = v_t / (1 - \beta_2^t)$
4.  **更新参数**：
    $$ \theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{\hat{v}_t} + \epsilon} \hat{m}_t $$
```


## AdamW

在深度学习中，我们常用 L2 正则化来防止过拟合，它的公式是：

$$ L_{total} = L_{loss} + \frac{\lambda}{2} \|\theta\|^2 $$

对它求梯度：
$$ \nabla L_{total} = \nabla L_{loss} + \lambda \theta $$

在 SGD 中，权重更新变成了：
$$ \theta_{t+1} = \theta_t - \eta (\nabla L_{loss} + \lambda \theta_t) = \theta_t - \eta \nabla L_{loss} - \eta \lambda \theta_t $$

最后这一项 $-\eta \lambda \theta_t$ 就是权重衰减 (Weight Decay)：每次更新时，都把权重 $\theta$ 缩小一点点（乘以 $1 - \eta\lambda$）。

但是传统的 Adam 实现是把 L2 正则项直接加到梯度 $g_t$ 里去算的：
$$ g_t \leftarrow \nabla L_{loss} + \lambda \theta_t $$

然后这个 $g_t$ 会被放入 $m_t$ 和 $v_t$ 的计算中，最后被 $\frac{1}{\sqrt{\hat{v}_t}}$ 缩放。这意味着：**权重的衰减力度被自适应学习率干扰了！**

*   如果某个参数的梯度变化很大（$v_t$ 大），它的权重衰减力度就会变小。
*   这导致 L2 正则化在 Adam 上效果不佳，不如 SGD。

```{note}
AdamW 修复了 Adam 中权重衰减 (Weight Decay) 的错误实现。它的做法非常直接：**解耦 (Decouple)**。它不把权重衰减项加到梯度里，而是在参数更新的最后一步，独立地减去权重衰减项：

$$ \theta_{t+1} = \text{AdamUpdate}(\theta_t, g_t) - \eta \lambda \theta_t $$
```
